In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [109]:
#블로그 url 크롤링
url_list = []
for i in range(1,8) :
    url2 = f'https://blog.naver.com/PostSearchList.naver?blogId=jjong0496&categoryNo=0&SearchText=%EA%B5%AD%EB%82%B4+%EA%B4%91%EA%B3%A0+%EC%B9%B4%ED%94%BC+%EB%AA%A8%EC%9D%8C&orderBy=date&term=&startDate=&endDate=&range=all&cpage={i}'

    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url2, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml") 

    links = soup.select('a.s_link')

    for l in links :
        link = l.get('href')
        url_list.append(link)
print(len(url_list))


63


In [2]:
#네이버 블로그 iframe 제거 함수
def delete_iframe(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml") 

    src_url = "https://blog.naver.com/" + soup.iframe["src"] #iframe 안에 있는 src 부분을 가져옴
    
    return src_url

In [3]:
#네이버 블로그 글 크롤링 함수
def text_scraping(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml") 
    
    if soup.select_one('div.se-main-container'):
        text = soup.select_one('div.se-main-container').text
        text = text.replace('\n','')
        #print("블로그")
        return text
    elif soup.select_one('div#postViewArea'):
        text = soup.select_one('div#postViewArea').text
        #text = text.replace('\n','')
        #print('블로그 2.0')
        return text
    else:
        return "확인불가"

In [182]:
#네이버 블로그 글 크롤링 
import requests
import re #추가
from bs4 import BeautifulSoup
from urllib.parse import quote

blog_list =[]
for i in url_list :
    post_link = i
    blog_text = text_scraping(delete_iframe(post_link))
    blog_list.append(blog_text)
    print('#',end='')
print('완료')


###############################################################완료


In [8]:
import re

def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

In [19]:
string = "abcdefㄱㄴㄷㄹㅁㅂ가나다라마바사12345[]{}().,!?'`~;:" 
re.sub('[^A-Za-z0-9가-힣]', '', string)



'abcdef가나다라마바사12345'

In [209]:
#회사, 문구, 카테고리 나누기
cnt = 0
copy_list =[]
company_list = []
category_list = []
for b in blog_list :
    print(cnt,end=',')
        
    text = b.split('<')
    text.pop(0) #처음 공백 제거



    for i in text : #카테고리 분류
        try :
            i = i.split('>')
            category = i[0]
            #print(category)
            copy_ = i[1].split(')')
            copy_.pop(len(copy_)-1) #마지막 공백 제거
        except :
            print('카테고리 오류',end='/')
            continue


        for j in copy_ : #copy 분류
            try :
                j= j.split('(')
                #print(j)
                copy = j[0]
                company = j[1]
                
                copy_list.append(copy)
                company_list.append(company)
                category_list.append(category)
            except : 
                print('인덱싱 오류',end='/')


    cnt += 1
print('나누기 완료')
df = pd.DataFrame({'company': company_list, 'copy' : copy_list, 'category' : category_list})
df.info()
df.to_csv('company_and_copy2.csv',encoding='utf-8-sig')
print('저장 완료')
        
        
    

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,인덱싱 오류/48,49,50,51,52,53,54,55,56,57,58,59,카테고리 오류/60,61,카테고리 오류/62,나누기 완료
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11053 entries, 0 to 11052
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   company   11053 non-null  object
 1   copy      11053 non-null  object
 2   category  11053 non-null  object
dtypes: object(3)
memory usage: 259.2+ KB
저장 완료


In [38]:
#슬로건 나누기
df = pd.read_csv('slogan_2.csv', index_col= None)
df.info()
df.head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1729 entries, 0 to 1728
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   company   1729 non-null   object
 1   copy      1729 non-null   object
 2   category  1729 non-null   object
dtypes: object(3)
memory usage: 40.6+ KB


<bound method NDFrame.head of                       company                     copy category
0         T’ESTIMO(가네보) 화장/세제            입술 글래머 LOOK!     화장/세제
1             노드(꽃을든남자) 화장/세제           피부만큼은 남자가 아니다     화장/세제
2      에뛰드 스타일 립스 아쿠아톡스 화장/세제               몰래몰래,입술水술     화장/세제
3        에뛰느 플라워 틴트 아이즈 화장/세제            사르르 꽃으로 물들이다     화장/세제
4              A.C.CARE 화장/세제    뾰루퉁한 피부, 과학으로 다독거리자!     화장/세제
...                       ...                      ...      ...
1724                    틱톡 기타    누구나 15초만에 스타가 될 수 있다        기타
1725                캐시슬라이드 기타            즐거운 적립생활의 시작        기타
1726                 스브스뉴스 기타            뉴스에는 위아래가 없다        기타
1727         홈플러스 신선 A/S센터 기타  1% 맘에 안 들어도 100% 교환,환불        기타
1728                 보건복지부 기타      우리는 지금 담배와의 전쟁중입니다        기타

[1729 rows x 3 columns]>

In [36]:
company = []
slogans = []
for slogan in df.slogan :
    slogan = slogan.replace(')','')
    slogan = slogan.split('(')
    slogans.append(slogan[0])
    company.append(slogan.pop(-1))

df2 = pd.DataFrame({'company': company, 'copy' : slogans, 'category' : df.category})

In [37]:
df2.to_csv('slogan_200.csv',encoding='utf-8-sig')

In [ ]:
#오류가 날 경우 확인 코드

In [195]:
print(url_list[59])

https://blog.naver.com/jjong0496?Redirect=Log&logNo=220776655881&from=postView


In [196]:
text = blog_list[59].split('<')
text.pop(0) #처음 공백 제거

copy_list =[]
company_list = []
category_list = []

for i in text : #카테고리 분류
    i = i.split('>')
    category = i[0]
    #print(category)
    copy_ = i[1].split(')')
    copy_.pop(len(copy_)-1) #마지막 공백 제거
    #print(copy_)    

    for j in copy_ : #copy 분류
        try :
            j= j.split('(')
            print(j)
            copy = j[0]
            company = j[1]
        except : 
            continue

        copy_list.append(copy)
        company_list.append(company)
        category_list.append(category)

['당신의 일주일은 할인으로 꽉 차야하니까요 ', '요기요']
['푸드득, 푸드를 득템하다 ', '동원몰']
['모바일 동영상의 신세계 ', '옥수수']
['디자인도 가격도 더 가볍게, 기본에 집중하다 ', 'LG X스킨']
['골라 골라 마음껏 골라 ', 'SBS고릴라팟']
['스크린의 감동을 가볍게 테이크아웃하다 ', 'LG X5']
['EVERYDAY ', '요기요']
['쇼핑이 생각날 땐, 어서옥션 ', '옥션']
['Simple Secure Virtually anywhere ', '삼성페이']
['여행을 다담다 ', 'g마켓']
['실감나게 재밌다 ', 'KT기가 IOT헬스']
['모든길은 T-MAP으로 통한다 ', 'T맵']
['우리, 살고 싶은 데서 살자 ', '다방']
['대한민국 넘버원 골프양말 ', '렉시']
['완벽은 없다 ', '리복 우먼스']
['SURPRICE JELLY, ICE JELLY ', '젤리벨리']
[' 가슴이 뚜껑뚜껑 ', '왕뚜껑']
['볶음 진짬뽕 외에는 절대 현혹되지마소 ', '볶음진짬뽕']
['특별한 커피는 파티를 부른다 ', '엔제리너스커피 아메리치노']
['월 화 수 목 금 토 일 식사하세요! ', '햇반컵반']
['황홀한 맛의 놀라움 ', '하겐다즈 초콜렛초코 아몬드']
['꽉꽉채워 딜리셔스 ', 'KFC매직박스']
['맛있는 피자는 작은 차이로부터 ', '피자헛']
['럭셔리 세단버거 ', '롯데리아 아재버거']
['라틴의 맛을 만나다 ', '맥도날드 리우1955버거']
['살아있는 유산균을 즐기는 새로운 방법 ', '요하이 유산균샌드']
['맛있게 쫄~깃 ', '젤리밥']
['유럽풍 퓨전 비빔면 ', '드레싱누들']
['시원한 가격으로 더위를 가격하라 ', '롯데리아 빙수']
['맛칼난다! ', '풀무원 육칼']
['보험은 그 시대의 목소리를 들어야 합니다 ', '메트라이프생명']
['오늘, 가장 푸르를 당신의 청춘을 위해 ', 'MG새마을금고']
['포인트도 자산이다 ', '신한카드 판 클럽']
['투명하

IndexError: list index out of range